<a href="https://colab.research.google.com/github/AntonioMoradoRamos/meia-masterdegree-experiments/blob/main/ML-Tarefas%5Ctarefas_ml_2_preparar_conjunto_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset preparation and generation

Ficheiro "data1.xlsx" (Full Year of historical data (1st data set) (53 907 KB) ).<br>
Importar o ficheiro para o colab.<br>

Gera um CSV com os dias do mês de Janeiro. Em cada dia contém registos em intervalos de 5 minutos desde a zero hora. Em que cada dia contém os valores das seguintes features:<br>
*   date
*   time
*   consumption (w)
*   generation (w)
*   temperature (˚C)	
*   humidity (%)
*   radiation (Wm^2)
*   isoweekday
*   period_of_day


Cada dia do mês está classificado nos respectivos dias da semana, sendo que:
*   Monday      -> 1
*   Tuesday     -> 2
*   Wednesday   -> 3
*   Thrusday    -> 4
*   Friday      -> 5
*   Saturday    -> 6
*   Sunday      -> 7

Classificação dos registos em períodos do dia:
*   madrugada: 00:05:00 -> 05:55:00 -> 1
*   manhã: 06:05:00 -> 11:55:00 -> 2
*   tarde: 12:05:00 -> 17:55:00 -> 3
*   noite: 18:05:00 -> 23:55:00 -> 4

Nome do ficheiro gerado: jan_2019_periodo_dia.csv


##Libraries

In [ ]:
import pandas as pd
import numpy as np

import datetime
from datetime import date, timedelta
from datetime import time

In [ ]:
# Versão do python
!python --version

Python 3.7.12


In [ ]:
# Versão do panas
print(pd.__version__)

1.1.5


##Load Dataset

In [ ]:
from google.colab import files
def export_data(ds, brandname):
  ds.to_csv(brandname, sep=';')
  files.download(brandname)

In [ ]:
import datetime
from datetime import date
def convert_int64_to_timestamp(dateAsInt64):
  #print("datetime as int64: " , dateAsInt64)
  theTimeStamp = pd.to_datetime(dateAsInt64, unit='s')
  #print("datetime as timestamp: " , theTimeStamp)

  return theTimeStamp



In [ ]:
# https://docs.python.org/3/library/datetime.html
#date.isoweekday()
# Return the day of the week as an integer, where Monday is 1 and Sunday is 7. For example, date(2002, 12, 4).isoweekday() == 3, a Wednesday. See also weekday(), isocalendar().
def get_isoweekday(theDate):
  return theDate.isoweekday()


In [ ]:
def get_day_period(theTime):
  
  madrugada = 1
  ini_madrugada = time(hour=0, minute=0, second=0, microsecond=0)
  fim_madrugada = time(hour=5, minute=55, second=0, microsecond=0)

  manha = 2
  ini_manha = time(hour=6, minute=0, second=0, microsecond=0)
  fim_manha = time(hour=11, minute=55, second=0, microsecond=0)
  
  tarde = 3
  ini_tarde = time(hour=12, minute=0, second=0, microsecond=0)
  fim_tarde = time(hour=17, minute=55, second=0, microsecond=0)
  
  noite = 4
  ini_noite = time(hour=18, minute=0, second=0, microsecond=0)
  fim_noite = time(hour=23, minute=55, second=0, microsecond=0)
  
  
  if theTime >= ini_madrugada and theTime <= fim_madrugada:
    #print("madrugada: ", theDate.time())
    return madrugada

  if theTime >= ini_manha and theTime <= fim_manha:
    #print("manhã: ", theDate.time())
    return manha

  if theTime >= ini_tarde and theTime <= fim_tarde:
    #print("tarde: ", theDate.time())
    return tarde

  if theTime >= ini_noite and theTime <= fim_noite:
    #print("noite: ", theDate.time())
    return noite

In [ ]:
def split_timestamp_to_date_and_time(ds, date_time_feature_name):
  series_date =  pd.to_datetime(convert_int64_to_timestamp(ds[date_time_feature_name]))
  # Para além de obter a data, i.e., o ano, o mês e o dia, é preciso garantir que o tipo de dados é datetime64
  # Este é o tipo de dados dos outros conjuntos de dados. 
  # É preciso que seja igual para realizar o merge
  ds['date'] = pd.to_datetime(pd.to_datetime(series_date).dt.date) 

  series_time =  pd.to_datetime(convert_int64_to_timestamp(ds[date_time_feature_name]))
  ds['time'] = pd.to_datetime(series_time).dt.time

  return ds

In [ ]:
##Merge dos dois conjuntos de dados utilizando as features "date" e "time"
def merge_ds_by_date_and_time(ds1, ds2):
  ds_merged = pd.merge(ds1, ds2, on=['date', 'time'])

  # Remover o datetime
  ds_merged.drop(columns=['datetime (epoch)'], inplace=True)
  
  return ds_merged

In [ ]:
def get_ds_by_day(ds, day):
  query = "date == " + day
  #print(query)
  ds_filter = ds.query(query)
  return ds_filter


In [ ]:
#def get_ds_mean_values_per_date(ds, date):
#  #date = '20190101'
#  #print(date)
#  ds_day = get_ds_by_day(ds, date)
#  gb1 = ds_day.groupby(['date']).mean()
#  # converter groupby para dataframew
#  ds_groupby = gb1.reset_index()
#  return ds_groupby

In [ ]:
#def get_ds_by_day(ds, day):
#  query = "date == " + day
#  #print(query)
#  ds_filter = ds.query(query)
#  return ds_filter

In [ ]:
def ds_mes(ds, periodo):
  return ds.query(periodo);
  

In [ ]:
def load_excel_data():
  return pd.ExcelFile('data1.xlsx')

In [ ]:
def prepare_data(xsls, periodo):
  ds_building_energy = pd.read_excel(xsls, 'building_energy')
  ds_weather_data = pd.read_excel(xsls, 'weather_data')

  ### Adicionar ao dataset "ds_weather_data" duas colunas
  # a) date 
  # b) time 
  #
  # Estas duas colunas são necessárias para o merge entre os três conjuntos de dados

  # No dataset ds_weather_data o registo no instante temporal é feito com o tipo de dados datetime
  ds_weather_data = split_timestamp_to_date_and_time(ds_weather_data, 'datetime (epoch)')

  ds_merged = merge_ds_by_date_and_time(ds_building_energy, ds_weather_data)

  ds_merged['isoweekday'] = ds_merged['date'].apply(get_isoweekday)
  ds_merged['period_of_day'] = ds_merged['time'].apply(get_day_period)
  ds_prepared = ds_mes(ds_merged, periodo)
  return ds_prepared

In [ ]:
xsls = load_excel_data()

In [ ]:
#ds.head()
jan = "date >= 20190101 and date <= 20190131"
file_name = "jan_2019_periodo_dia.csv"

ds_mes = prepare_data(xsls, jan)
export_data(ds_mes, file_name)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
ds_mes.head()

,date,time,consumption (w),generation (w),temperature (˚C),humidity (%),radiation (Wm^2),isoweekday,period_of_day
0,2019-01-01,00:05:00,2985,0,8.7,76.0,0.0,2,1
1,2019-01-01,00:10:00,2258,0,8.6,76.0,0.0,2,1
2,2019-01-01,00:15:00,2266,0,8.7,76.0,0.0,2,1
3,2019-01-01,00:20:00,3016,0,8.7,76.0,0.0,2,1
4,2019-01-01,00:25:00,2265,0,8.6,76.0,0.0,2,1


In [ ]:
ds_mes.tail()

,date,time,consumption (w),generation (w),temperature (˚C),humidity (%),radiation (Wm^2),isoweekday,period_of_day
8922,2019-01-31,23:35:00,2917,0,11.7,80.0,0.0,4,4
8923,2019-01-31,23:40:00,3990,0,11.8,79.0,0.0,4,4
8924,2019-01-31,23:45:00,3377,0,11.7,80.0,0.0,4,4
8925,2019-01-31,23:50:00,2647,0,11.2,81.0,0.0,4,4
8926,2019-01-31,23:55:00,4228,0,10.9,86.0,0.0,4,4


In [ ]:
ds_mes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8927 entries, 0 to 8926
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              8927 non-null   datetime64[ns]
 1   time              8927 non-null   object        
 2   consumption (w)   8927 non-null   int64         
 3   generation (w)    8927 non-null   int64         
 4   temperature (˚C)  8927 non-null   float64       
 5   humidity (%)      8927 non-null   float64       
 6   radiation (Wm^2)  8927 non-null   float64       
 7   isoweekday        8927 non-null   int64         
 8   period_of_day     8927 non-null   int64         
dtypes: datetime64[ns](1), float64(3), int64(4), object(1)
memory usage: 697.4+ KB


#Bibliografia

*   Dataset gecad - smartgridcompetitions<br>
http://www.gecad.isep.ipp.pt/smartgridcompetitions/data/

*   Python<br>
https://docs.python.org/3.7/library/datetime.html#module-datetime<br>
https://docs.python.org/3.7/library/datetime.html#datetime.datetime<br>
https://docs.python.org/3/library/datetime.html<br>
https://estatsite.com.br/2019/04/30/join-merge-no-python-pandas/<br>
https://medium.com/analytics-vidhya/how-to-pd-merge-two-data-frames-on-a-common-date-column-e7808d7ccaee<br>
https://www.analyticsvidhya.com/blog/2020/05/datetime-variables-python-pandas/



*   Pandas<br>
https://pandas.pydata.org/docs/getting_started/index.html#getting-started<br>
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html<br>
https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.date.html?highlight=date#pandas.Series.dt.date<br>
https://pandas.pydata.org/docs/reference/api/pandas.Series.apply.html<br>
https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html<br>
https://stackoverflow.com/questions/26521266/using-pandas-to-pd-read-excel-for-multiple-worksheets-of-the-same-workbook<br>
https://www.delftstack.com/pt/howto/python-pandas/pandas-unique-values-in-column/<br>

